In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [17]:
page = requests.get('https://www.airlinequality.com/airline-reviews/british-airways/?sortby=post_date%3ADesc&pagesize=100')

In [18]:
page.content

b'<!doctype html>\n\n<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7 lt-ie10" lang="en-GB"> <![endif]-->\n<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8 lt-ie10" lang="en-GB"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js lt-ie9 lt-ie10" lang="en-GB"> <![endif]-->\n<!--[if IE 9]>    <html class="no-js lt-ie10" lang="en-GB"> <![endif]-->\n<!--[if gt IE 8]><!-->\n<html lang="en-GB">\n<!--<![endif]-->\n\n<head>\n    <meta charset="utf-8">\n\n    <title>British Airways Customer Reviews - SKYTRAX</title>\n\n    <!-- Google Chrome Frame for IE -->\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n\n    <!-- mobile meta -->\n    <meta name="HandheldFriendly" content="True">\n    <meta name="MobileOptimized" content="320">\n    <meta name="viewport"\n        content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no" />\n    <!-- icons & favicons -->\n    <link rel="apple-touch-icon" href="https://www.airlinequal

In [19]:
soup = BeautifulSoup(page.text, 'html.parser')

In [20]:
last_page=int([item.text for item in soup.find(class_='comp comp_reviews-pagination querylist-pagination position-').find_all('a')][-2])

In [21]:
links = ['https://www.airlinequality.com/airline-reviews/british-airways/page/'+str(i)+\
         '/?sortby=post_date%3ADesc&pagesize=100' for i in range(1, last_page+1)]

In [22]:
links

['https://www.airlinequality.com/airline-reviews/british-airways/page/1/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/airline-reviews/british-airways/page/2/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/airline-reviews/british-airways/page/3/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/airline-reviews/british-airways/page/4/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/airline-reviews/british-airways/page/5/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/airline-reviews/british-airways/page/6/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/airline-reviews/british-airways/page/7/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/airline-reviews/british-airways/page/8/?sortby=post_date%3ADesc&pagesize=100',
 'https://www.airlinequality.com/airline-reviews/british-airways/page/9/?sortby=post_date%3ADesc&pagesiz

In [23]:
collected_data=[]

In [27]:
for link in links:
    req = requests.get(link, timeout=60)
    
    soup = BeautifulSoup(req.text, 'html.parser')

    current_page=soup.find_all('article', attrs={'itemprop':'review'})

    for content in current_page:

        dictionary={}

        date=content.find( attrs={'itemprop':'datePublished'}).get('content') #get the date
        dictionary['Date']=date

        rating=content.find(class_='rating-10').text.strip() #get the rating
        dictionary['Rating']=rating

        review_heading=content.find(class_='text_header').text #get the review heading
        dictionary['Review Heading']= review_heading

        user_info=content.find(class_='text_sub_header userStatusWrapper').text  #get user info
        dictionary['User info'] = user_info

        review_text=content.find(class_='text_content').text #get the review text
        dictionary['Review'] = review_text



        review_stats=content.find(class_='review-ratings').find_all('tr') #get review stats

        #organise the review stats

        for row in review_stats:
            cells = row.find_all('td')


            row_data  = [cell for cell in cells]


            if row_data[1].find_all(class_='star fill') != []:
                row_data[1]=row_data[1].find_all(class_='star fill')[-1]

            row_data = [cell.text for cell in row_data]


            dictionary[row_data[0]]=row_data[1]


        collected_data.append(dictionary)

In [30]:
data=pd.DataFrame(collected_data)

In [32]:
data.to_csv('British Airway Review.csv', index=False)